# Industry Classifications and Crosswalks

Although ACS supposedly has an NAICS industry variable, it doesn't gel with the summary-level industries provided in the IO Tables from the BEA.

This notebook discusses how these differences were reconciled.

#### Computer-Specific Code Setup

Following lines should be run once. You may need to change this if you're not running this on MW's machine.

In [408]:
# May need to reset working directory
# Used for relative paths below
import os 
wkdir = '/home/michael/Dropbox/grantThornton/projects/tariffsTrump/code/py'
os.chdir(wkdir)

In [409]:
import pandas as pd
import numpy as np
import re,warnings

from src.IpumsExtract import IpumsExtract


## Sorting through the ACS classification

In [410]:
NAICS_FILE = '../data/raw/naics_codes_ipums.xlsx'
NAICS_SHEET = 'data'

acs = pd.read_excel(NAICS_FILE, sheetname=NAICS_SHEET)
acs.rename(
columns={
  'INDNAICS CODE\n(2003-onward ACS/PRCS)': 'ind',
  '2012 NAICS EQUIVALENT': 'naics',
  'INDUSTRY TITLE': 'desc'
},
inplace=True
)
acs = acs[['ind', 'naics', 'desc']]

acs.dropna(how='all', axis=0, inplace=True)
acs.fillna(method='ffill', inplace=True)

Here's a look at the data:

In [411]:
acs

,ind,naics,desc
1,0,-,N/A (less than 16 years old/unemployed who nev...
3,111,111,Crop production
4,112,112,Animal production (2003-2012)
5,112,112,Animal production and aquaculture (2013-onward)
6,113M,"1131, 1132","Forestry, except logging"
7,1133,1133,Logging
8,114,114,"Fishing, hunting, and trapping"
9,115,115,Support activities for agriculture and forestry
11,211,211,Oil and gas extraction
12,2121,2121,Coal mining


We will:


- In the `ind` variable:
    - Drop at `ind`=0 because these people are not in the ACS dataset or the IO table.
    - Drop cases outside of 2013- (`'51M (2003-2007)'`)
    - Reformat cases: `'3231 (2008-onward)'`
    - Reformat all to strings
    - Remove hidden characters from strings (`\n`)
- Deal with several mapping cases:
     - Direct: e.g. `213 => 213`
     - Multiple: e.g. `213 => 213, 213s`
     - "Part of": e.g. `213 => Part of 21`
     - All except: e.g. `3118Z	=> 3118 exc. 311811` or `928P => '928 (exc. 928110)'` or `3329 exc. 332992-332995`
     - Combinations of above: e.g. `221MP =>	Pts. 2211, 2212`

#### `ind` Variable Reformatting

In [412]:
# Get rid of unemployed/NA
acs = acs.loc[acs['naics'] != '-'].copy()

# Convert all to strings
acs['ind'] = acs['ind'].apply(str)

Now go to _discontinued series_. All of these discontinued series were replaced by others with the same mapping.

In [413]:
# Discontinued series
discon = [ind for ind in acs['ind'] if re.match('(.*)([\\n]?)\(2003\-20[107824]\)',ind) is None]
# discon = [ind for ind in discon if re.match('(.*)\(2003\-2007\)',ind) is None]

acs = acs.loc[acs['ind'].isin(discon)]

In [414]:
acs['ind'] = acs['ind'].apply(lambda d: re.sub('(.*)(\s+)(\(20[05813]+\-onward\))', '\\1', d))

Now we need to handle the special case of 8114. We will re-code this manually:

In [415]:
acs.loc[acs['ind'] == '8114Z (8114 in 2005-onward. Incl. 81143 in 2005-onward.)', ['ind', 'naics']] = ['8114', '8114, 81143']

In [416]:
acs = acs.loc[acs['ind'] != '81143 (pt. of 8114 in 2005-onward.)'].copy()

_Check._

Now we need to double-check that all of the industries found in the ACS data are also found in the cross-walk.

In [417]:
# Initiate IpumsExtract Object
ie = IpumsExtract(
  '../data/raw/usa_00022.dat.gz',
  '../data/raw/usa_00022.do',
  db_filename='../data/int/acs_2016.db'
)

# Get the number of workers in each county-industry
script = """
SELECT DISTINCT indnaics as ind
FROM
  main
;
"""

check = ie.read_sql(script)

In [418]:
test = all([cind in acs['ind'].values for cind in check.values])
if not test:
    not_in_acs = list([cind for cind in check.values if (cind not in acs['ind'].values)][0])
    warnings.warn('Industries not found in ACS: {}'.format(not_in_acs))

/home/michael/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Industries not found in ACS: ['333MS']
  after removing the cwd from sys.path.


It appears that one industry is missing. After looking at the source data it is clear that `333MS` is not in the original crosswalk data.

I will append `333MS` to the `acs` frame and say that it corresponds to the same values as `333M` ('Machinery, n.e.c.').

In [419]:
acs = (
  acs.append(
    pd.DataFrame(
      [
        {
          'ind': '333MS',
          'naics': '3332, 3334, 3339',
          'desc': 'Machinery, n.e.c.'
        }
      ]
    )
  ).reset_index(drop=True)
)

#### Duplicate `ind` due to `desc` revision

There are certain cases where there are duplicate industries. From the below, it is clear that this is due to industry renamings (and not different mappings to to the NAICS).

In each case we'll keep the last description.

In [420]:
dd = acs['ind'].duplicated('last')
acs.loc[dd == True]

,desc,ind,naics
1,Animal production (2003-2012),112,112
26,"Bakeries, except retail (2003-2012)",3118Z,3118 exc. 311811
37,Knitting mills (2003-2007),31M,"31324, 3151"
42,"Leather tanning and products, except footwear ...",316M,"3161, 3169"
45,Paperboard containers and boxes (2003-2012),32221,32221
61,"Pottery, ceramics, and related products (2003-...",32711,32711
74,"Structural metals, and tank and shipping conta...",332M,"3323, 3324"
82,Construction mining and oil field machinery (2...,3331M,"33312, 33313"
94,"Electrical machinery, equipment, and supplies,...",335M,"3351, 3353, 3359"
106,Furniture and fixtures (2003-2007),337,337


In [422]:
acs = acs.loc[~dd].copy()

## Linking to Industries in the IO Tables

As discussed above, the IO Tables follow a different summarization of the full NAICS scheme than the ACS does. As a result, the mapping will not be one-for-one.

After some tinkering I figured that the fastest way to do this would be by hand. There are simply too many quirks in the data to do this easily by code. Almost all of the crossing was easy/straightforward to do by hand, though.

In [443]:
acs['io_ind'] = [[] for ii in range( len(acs))]

In [425]:
acs.index = acs['ind']

In [444]:
acs.at["111", 'io_ind'] = [
  "1111A0", "1111B0", "111200", "111300", "111400", "111900"
]
acs.at["112", 'io_ind'] = ["1121A0", "112120", "112A00", "112300"]
acs.at["113M", 'io_ind'] = [113000]
acs.at["1133", 'io_ind'] = [113000]
acs.at["114", 'io_ind'] = [114000]
acs.at["115", 'io_ind'] = [115000]
acs.at["211", 'io_ind'] = [211000]
acs.at["2121", 'io_ind'] = [212100]
acs.at["2122", 'io_ind'] = ["2122A0", "212230"]
acs.at["2123", 'io_ind'] = ["212310", "2123A0"]
acs.at["21S", 'io_ind'] = [
  "211000", "212100", "2122A0", "212230", "212310", "2123A0", "213111",
  "21311A"
]
acs.at["213", 'io_ind'] = ["213111", "21311A"]
acs.at["2211P", 'io_ind'] = [221100]
acs.at["2212P", 'io_ind'] = [221200]
acs.at["221MP", 'io_ind'] = [221200, 221100, "S00202"]
acs.at["2213M", 'io_ind'] = [221300]
acs.at["22132", 'io_ind'] = [221300]
acs.at["22S", 'io_ind'] = [221200, 221100, 221300]
acs.at["23", 'io_ind'] = [
  "230301", "230302", "233210", "233230", "233240", "233262", "233293",
  "2332A0", "2332B0", "233411", "233412", "2334A0"
]
acs.at["311M1", 'io_ind'] = [
  "311111", "311119", "311210", "311221", "31122A", "311225", "311230"
]
acs.at["3113", 'io_ind'] = [311300]
acs.at["3114", 'io_ind'] = [311410, 311420]
acs.at["3115", 'io_ind'] = ["31151A", 311513, 311514, 311520]
acs.at["3116", 'io_ind'] = ["31161A", 311615]
acs.at["311811", 'io_ind'] = [311810]
acs.at["3118Z", 'io_ind'] = ["3118A0"]
acs.at["311M2", 'io_ind'] = [311700, 311910, 311920, 311930, 311940, 311990]
acs.at["311S", 'io_ind'] = [
  311111, 311119, 311210, 311221, "31122A", 311225, 311230, 311300, 311410,
  311420, "31151A", 311513, 311514, 311520, "31161A", 311615, 311700, 311810,
  "3118A0", 311910, 311920, 311930, 311940, 311990, 312110, 312120, 312130,
  312140, 312200
]
acs.at["3121", 'io_ind'] = [312110, 312120, 312130, 312140]
acs.at["3122", 'io_ind'] = [312200]
acs.at["3131", 'io_ind'] = [313100]
acs.at["3132Z", 'io_ind'] = [313200]
acs.at["3133", 'io_ind'] = [313300]
acs.at["31411", 'io_ind'] = [314110]
acs.at["314Z", 'io_ind'] = [314120, 314900]
acs.at["31M", 'io_ind'] = [313200]
acs.at["3152", 'io_ind'] = [315000]
acs.at["3159", 'io_ind'] = [315000]
acs.at["3162", 'io_ind'] = [316000]
acs.at["316M", 'io_ind'] = [316000]
acs.at["3221", 'io_ind'] = [322110, 322120, 322130]
acs.at["32221", 'io_ind'] = [322210, 322220, 322230, 322291]
acs.at["3222M", 'io_ind'] = [322299]
# acs.at["323 (2003-2007)", 'io_ind'] = -99
acs.at["3231", 'io_ind'] = [323110, 323120]
acs.at["32411", 'io_ind'] = [324110]
acs.at["3241M", 'io_ind'] = [324121, 324122, 324190]
acs.at["3252", 'io_ind'] = [325211, "3252A0"]
acs.at["3253", 'io_ind'] = [325310, 325320]
acs.at["3254", 'io_ind'] = [325411, 325412, 325413, 325414]
acs.at["3255", 'io_ind'] = [325510, 325520]
acs.at["3256", 'io_ind'] = [325610, 325620]
acs.at["325M", 'io_ind'] = [
  325910, "3259A0", 325110, 325120, 325130, 325180, 325190
]
acs.at["3261", 'io_ind'] = [
  326110, 326120, 326130, 326140, 326150, 326160, 326190
]
acs.at["32621", 'io_ind'] = [326210]
acs.at["3262M", 'io_ind'] = [326220, 326290]
acs.at["32711", 'io_ind'] = [327100]
# acs.at["32712 (2003-2012)", 'io_ind'] = -99
acs.at["327120", 'io_ind'] = [327100]
acs.at["3272", 'io_ind'] = [327200]
acs.at["327M", 'io_ind'] = [327310, 327320, 327330, 327390, 327400]
acs.at["3279", 'io_ind'] = [327910, 327991, 327992, 327993, 327999]
acs.at["331M", 'io_ind'] = [331110, 331200]
acs.at["3313", 'io_ind'] = ["33131A", "331314", "33131B"]
acs.at["3314", 'io_ind'] = [331411, 331419, 331420, 331490]
acs.at["3315", 'io_ind'] = [331510, 331520]
acs.at["3321", 'io_ind'] = ["33211A", "332114", "33211B"]
acs.at["3322", 'io_ind'] = [332200]
acs.at["332M", 'io_ind'] = [332310, 332320, 332410, 332420, 332430]
acs.at["3327", 'io_ind'] = [332710, 332720]
acs.at["3328", 'io_ind'] = [332800]
acs.at["33299M", 'io_ind'] = ["332991", "33299A", "332996", "33299B"]
acs.at["332MZ", 'io_ind'] = [332500, 332600]
acs.at["33MS", 'io_ind'] = [
  "33211A", 332114, "33211B", 332200, 332310, 332320, 332410, 332420, 332430,
  332500, 332600, 332710, 332720, 332800, "33291A", 332913, 332991, "33299A",
  332996, "33299B"
]
acs.at["33311", 'io_ind'] = [333111]
acs.at["3331M", 'io_ind'] = [333120, 333130]
acs.at["3333", 'io_ind'] = ["33331A", 333313, 333314, 333315]
acs.at["3335", 'io_ind'] = [333511, "33351A", 333514, "33351B"]
acs.at["3336", 'io_ind'] = [333611, 333612, 333613, 333618]
acs.at["333M", 'io_ind'] = [
  "33329A", 333220, 333295, "33341A", 333414, 333415, "33391A", 333912, 333920,
  "333991", "33399A", 333993, 333994, "33399B"
]
acs.at["333S", 'io_ind'] = [
  333111, 333112, 333120, 333130, "33329A", 333220, 333295, "33331A", 333313,
  333314, 333315, "33341A", 333414, 333415, 333511, "33351A", 333514, "33351B",
  333611, 333612, 333613, 333618, "33391A", 333912, 333920, 333991, "33399A",
  333993, 333994, "33399B"
]
acs.at["3341", 'io_ind'] = [334111, 334112, "33411A"]
acs.at["334M1", 'io_ind'] = [334210, 334220, 334290, 334300]
acs.at["3345", 'io_ind'] = [
  334510, 334511, 334512, 334513, 334514, 334515, 334516, 334517, "33451A"
]
acs.at["334M2", 'io_ind'] = ["33441A", 334413, 334418, "334610"]
acs.at["3352", 'io_ind'] = [335210, 335221, 335222, 335224, 335228]
acs.at["335M", 'io_ind'] = [
  335110, 335120, 335311, 335312, 335313, 335314, 335911, 335912, 335920,
  335930, 335991, 335999
]
acs.at["336M", 'io_ind'] = [
  336111, 336112, 336120, 336211, 336212, 336213, 336214, 336310, 336320,
  "3363A0", 336350, 336360, 336370, 336390
]
acs.at["33641M1", 'io_ind'] = [336411, 336412, 336413]
acs.at["33641M2", 'io_ind'] = [336414, "33641A"]
acs.at["3365", 'io_ind'] = [336500]
acs.at["3366", 'io_ind'] = [336611, 336612]
acs.at["3369", 'io_ind'] = [336991, 336992, 336999]
acs.at["3211", 'io_ind'] = [321100]
acs.at["3212", 'io_ind'] = [321200]
acs.at["32199M", 'io_ind'] = [321910, "3219A0"]
acs.at["3219ZM", 'io_ind'] = ["3219A0"]
acs.at["337", 'io_ind'] = [
  337110, 337121, 337122, "33712A", 337127, "33721A", 337215, 337900
]
acs.at["3391", 'io_ind'] = [339112, 339113, 339114, 339115, 339116]
acs.at["3399M", 'io_ind'] = [339920, 339930]
acs.at["3399ZM", 'io_ind'] = [339910, 339940, 339950, 339990]
acs.at["3MS", 'io_ind'] = [
  "321100", "321200", "321910", "3219A0", "327100", "327200", "327310",
  "327320", "327330", "327390", "327400", "327910", "327991", "327992",
  "327993", "327999", "331110", "331200", "33131A", "331314", "33131B",
  "331411", "331419", "331420", "331490", "331510", "331520", "33211A",
  "332114", "33211B", "332200", "332310", "332320", "332410", "332420",
  "332430", "332500", "332600", "332710", "332720", "332800", "33291A",
  "332913", "332991", "33299A", "332996", "33299B", "333111", "333112",
  "333120", "333130", "33329A", "333220", "333295", "33331A", "333313",
  "333314", "333315", "33341A", "333414", "333415", "333511", "33351A",
  "333514", "33351B", "333611", "333612", "333613", "333618", "33391A",
  "333912", "333920", "333991", "33399A", "333993", "333994", "33399B",
  "334111", "334112", "33411A", "334210", "334220", "334290", "334300",
  "33441A", "334413", "334418", "334510", "334511", "334512", "334513",
  "334514", "334515", "334516", "334517", "33451A", "334610", "335110",
  "335120", "335210", "335221", "335222", "335224", "335228", "335311",
  "335312", "335313", "335314", "335911", "335912", "335920", "335930",
  "335991", "335999", "336111", "336112", "336120", "336211", "336212",
  "336213", "336214", "336310", "336320", "3363A0", "336350", "336360",
  "336370", "336390", "336411", "336412", "336413", "336414", "33641A",
  "336500", "336611", "336612", "336991", "336992", "336999", "337110",
  "337121", "337122", "33712A", "337127", "33721A", "337215", "337900",
  "339112", "339113", "339114", "339115", "339116", "339910", "339920",
  "339930", "339940", "339950", "339990", "311111", "311119", "311210",
  "311221", "31122A", "311225", "311230", "311300", "311410", "311420",
  "31151A", "311513", "311514", "311520", "31161A", "311615", "311700",
  "311810", "3118A0", "311910", "311920", "311930", "311940", "311990",
  "312110", "312120", "312130", "312140", "312200", "313100", "313200",
  "313300", "314110", "314120", "314900", "315000", "316000", "322110",
  "322120", "322130", "322210", "322220", "322230", "322291", "322299",
  "323110", "323120", "324110", "324121", "324122", "324190", "325110",
  "325120", "325130", "325180", "325190", "325211", "3252A0", "325310",
  "325320", "325411", "325412", "325413", "325414", "325510", "325520",
  "325610", "325620", "325910", "3259A0", "326110", "326120", "326130",
  "326140", "326150", "326160", "326190", "326210", "326220", "326290"
]
acs.at["4231", 'io_ind'] = [420000]
acs.at["4232", 'io_ind'] = [420000]
acs.at["4233", 'io_ind'] = [420000]
acs.at["4234", 'io_ind'] = [420000]
acs.at["4235", 'io_ind'] = [420000]
acs.at["4236", 'io_ind'] = [420000]
acs.at["4237", 'io_ind'] = [420000]
acs.at["4238", 'io_ind'] = [420000]
acs.at["42393", 'io_ind'] = [420000]
acs.at["4239Z", 'io_ind'] = [420000]
acs.at["4241", 'io_ind'] = [420000]
acs.at["424M", 'io_ind'] = [420000]
acs.at["4243", 'io_ind'] = [420000]
acs.at["4244", 'io_ind'] = [420000]
acs.at["4245", 'io_ind'] = [420000]
acs.at["4247", 'io_ind'] = [420000]
acs.at["4248", 'io_ind'] = [420000]
acs.at["42491", 'io_ind'] = [420000]
acs.at["4249Z", 'io_ind'] = [420000]
acs.at["4251", 'io_ind'] = [420000]
acs.at["42S", 'io_ind'] = [420000]
acs.at["4411", 'io_ind'] = [441000]
acs.at["4412", 'io_ind'] = [441000]
acs.at["4413", 'io_ind'] = [441000]
acs.at["442", 'io_ind'] = ["4A0000"]
acs.at["443111 (2003-2012)", 'io_ind'] = ["4A0000"]
acs.at["443141", 'io_ind'] = ["4A0000"]
acs.at["443142", 'io_ind'] = ["4A0000"]
acs.at["4441Z", 'io_ind'] = ["4A0000"]
acs.at["44413", 'io_ind'] = ["4A0000"]
acs.at["4442", 'io_ind'] = ["4A0000"]
acs.at["4451", 'io_ind'] = [445000]
acs.at["4452", 'io_ind'] = [445000]
acs.at["4453", 'io_ind'] = [445000]
acs.at["44611", 'io_ind'] = ["4A0000"]
acs.at["446Z", 'io_ind'] = ["4A0000"]
acs.at["447", 'io_ind'] = ["4A0000"]
acs.at["448ZM (2003-2007)", 'io_ind'] = ["4A0000"]
acs.at["4481", 'io_ind'] = ["4A0000"]
acs.at["44821", 'io_ind'] = ["4A0000"]
acs.at["4483", 'io_ind'] = ["4A0000"]
acs.at["4511M", 'io_ind'] = ["4A0000"]
acs.at["45113", 'io_ind'] = ["4A0000"]
acs.at["45114", 'io_ind'] = ["4A0000"]
acs.at["45121", 'io_ind'] = ["4A0000"]
acs.at["45211", 'io_ind'] = [452000]
acs.at["4529", 'io_ind'] = [452000]
acs.at["4531", 'io_ind'] = ["4A0000"]
acs.at["45321", 'io_ind'] = ["4A0000"]
acs.at["4533", 'io_ind'] = ["4A0000"]
acs.at["45322", 'io_ind'] = ["4A0000"]
acs.at["4539", 'io_ind'] = ["4A0000"]
acs.at["4541 (2003-2004)", 'io_ind'] = ["4A0000"]
acs.at["454111", 'io_ind'] = ["4A0000"]
acs.at["454112", 'io_ind'] = ["4A0000"]
acs.at["454113", 'io_ind'] = ["4A0000"]
acs.at["4542", 'io_ind'] = ["4A0000"]
acs.at["45431 (2003-2012)", 'io_ind'] = ["4A0000"]
acs.at["454310", 'io_ind'] = ["4A0000"]
acs.at["45439", 'io_ind'] = ["4A0000"]
acs.at["4MS", 'io_ind'] = [441000, 445000, 452000, "4A0000"]
acs.at["481", 'io_ind'] = [481000]
acs.at["482", 'io_ind'] = [482000]
acs.at["483", 'io_ind'] = [483000]
acs.at["484", 'io_ind'] = [484000]
acs.at["485M", 'io_ind'] = [485000, "S00201"]
acs.at["4853", 'io_ind'] = [485000]
acs.at["486", 'io_ind'] = [486000]
acs.at["487", 'io_ind'] = ["48A000"]
acs.at["488", 'io_ind'] = ["48A000"]
acs.at["491", 'io_ind'] = [491000]
acs.at["492", 'io_ind'] = [492000]
acs.at["493", 'io_ind'] = [493000]
acs.at["51111", 'io_ind'] = [511110]
acs.at["5111Z", 'io_ind'] = [511120, 511130, "5111A0"]
acs.at["5112", 'io_ind'] = [511200]
acs.at["5121", 'io_ind'] = [512100]
acs.at["5122", 'io_ind'] = [512200]
# acs.at["51M (2003-2007)", 'io_ind'] = -99
acs.at["515", 'io_ind'] = [515100, 515200]
acs.at["51913", 'io_ind'] = [519130]
acs.at["5171", 'io_ind'] = [517110]
acs.at["517Z", 'io_ind'] = [517210, "517A00"]
acs.at["5182", 'io_ind'] = [518200]
acs.at["51912", 'io_ind'] = ["5191A0"]
acs.at["5191Z (2003-2007)", 'io_ind'] = ["5191A0"]
acs.at["5191ZM", 'io_ind'] = ["5191A0"]
acs.at["52M1", 'io_ind'] = ["52A000"]
acs.at["5221M", 'io_ind'] = ["52A000"]
acs.at["522M", 'io_ind'] = ["522A00"]
acs.at["52M2", 'io_ind'] = ["523A00", 523900, 525000]
acs.at["524", 'io_ind'] = [524100, 524200]
acs.at["531", 'io_ind'] = ["5310HS", "531ORE"]
acs.at["5321", 'io_ind'] = [532100]
acs.at["53223", 'io_ind'] = ["532A00"]
acs.at["532M", 'io_ind'] = ["532A00"]
acs.at["53M", 'io_ind'] = [532400]
acs.at["5411", 'io_ind'] = [541100]
acs.at["5412", 'io_ind'] = [541200]
acs.at["5413", 'io_ind'] = [541300]
acs.at["5414", 'io_ind'] = [541400]
acs.at["5415", 'io_ind'] = [541511, 541512, "54151A"]
acs.at["5416", 'io_ind'] = [541610, "5416A0"]
acs.at["5417", 'io_ind'] = [541700]
acs.at["5418", 'io_ind'] = [541800]
acs.at["54194", 'io_ind'] = [541940]
acs.at["5419Z", 'io_ind'] = ["5419A0", 541920]
acs.at["55", 'io_ind'] = [550000]
acs.at["5613", 'io_ind'] = [561300]
acs.at["5614", 'io_ind'] = [561400]
acs.at["5615", 'io_ind'] = [561500]
acs.at["5616", 'io_ind'] = [561600]
acs.at["5617Z", 'io_ind'] = [561700]
acs.at["56173", 'io_ind'] = [561700]
acs.at["561M", 'io_ind'] = [561100, 561200, 561900]
acs.at["562", 'io_ind'] = [562000]
acs.at["6111", 'io_ind'] = [611100]
acs.at["611M1", 'io_ind'] = ["611A00"]
acs.at["611M2", 'io_ind'] = ["611B00"]
acs.at["611M3", 'io_ind'] = ["611B00"]
acs.at["6211", 'io_ind'] = [621100]
acs.at["6212", 'io_ind'] = [621200]
acs.at["62131", 'io_ind'] = [621300]
acs.at["62132", 'io_ind'] = [621300]
acs.at["6213ZM", 'io_ind'] = [621300]
acs.at["6214", 'io_ind'] = [621400]
acs.at["6216", 'io_ind'] = [621600]
acs.at["621M", 'io_ind'] = [621500, 621900]
acs.at["622", 'io_ind'] = [622000]
acs.at["6231", 'io_ind'] = ["623A00"]
acs.at["623M", 'io_ind'] = ["623B00", "623B00"]
acs.at["6241", 'io_ind'] = [624100]
acs.at["6242", 'io_ind'] = ["624A00"]
acs.at["6243", 'io_ind'] = ["624A00"]
acs.at["6244", 'io_ind'] = [624400]
acs.at["711", 'io_ind'] = [711100, 711200, "711A00", 711500]
acs.at["712", 'io_ind'] = [712000]
acs.at["71395", 'io_ind'] = [713900]
acs.at["713Z", 'io_ind'] = [713100, 713200, 713900]
acs.at["7211", 'io_ind'] = [721000]
acs.at["721M", 'io_ind'] = [721000]
acs.at["722Z", 'io_ind'] = [722110, 722211, "722A00"]
acs.at["7224", 'io_ind'] = ["722A00"]
acs.at["8111Z", 'io_ind'] = [811100]
acs.at["811192", 'io_ind'] = [811100]
acs.at["8112", 'io_ind'] = [811200]
acs.at["8113", 'io_ind'] = [811300]
acs.at["8114", 'io_ind'] = [811400]
acs.at["812111", 'io_ind'] = [812100]
acs.at["812112", 'io_ind'] = [812100]
acs.at["8121M", 'io_ind'] = [812100]
acs.at["8123", 'io_ind'] = [812300]
acs.at["8122", 'io_ind'] = [812200]
acs.at["8129", 'io_ind'] = [812900]
acs.at["8131", 'io_ind'] = [813100]
acs.at["813M", 'io_ind'] = ["813A00"]
acs.at["81393", 'io_ind'] = ["813B00"]
acs.at["8139Z", 'io_ind'] = ["813B00"]
acs.at["814", 'io_ind'] = [814000]
acs.at["9211MP", 'io_ind'] = ["S00600"]
acs.at["92113", 'io_ind'] = ["S00600"]
acs.at["92119", 'io_ind'] = ["S00600"]
acs.at["92MP", 'io_ind'] = ["S00600"]
acs.at["923", 'io_ind'] = ["S00600"]
acs.at["92M1", 'io_ind'] = ["S00600"]
acs.at["92M2", 'io_ind'] = ["S00600"]
acs.at["928P", 'io_ind'] = ["S00500"]
acs.at["928110P1", 'io_ind'] = ["S00500"]
acs.at["928110P2", 'io_ind'] = ["S00500"]
acs.at["928110P3", 'io_ind'] = ["S00500"]
acs.at["928110P4", 'io_ind'] = ["S00500"]
acs.at["928110P5", 'io_ind'] = ["S00500"]
acs.at["928110P6", 'io_ind'] = ["S00500"]
acs.at["928110P7", 'io_ind'] = ["S00500"]
acs.at["333MS", 'io_ind'] = [
  "33329A", 333220, 333295, "33341A", 333414, 333415, "33391A", "333912",
  "333920", "333991", "33399A", "333993", "333994", "33399B"
]


Did we miss any industries?

In [447]:
acs.loc[acs['io_ind'].apply(len)==0]

,desc,ind,naics,io_ind
ind,,,,
323 (2003-2007),Printing and related support activities,323 (2003-2007),3231,[]
32712 (2003-2012),Structural clay products (2003-2012),32712 (2003-2012),327120,[]
51M (2003-2007),Radio and television broadcasting and cable (2...,51M (2003-2007),515,[]


Appears only to be the industries that we want to drop anyway.

In [449]:
acs = acs.loc[acs['io_ind'].apply(len)!=0].copy()
acs.reset_index(drop=True, inplace=True)

Let's make everything a string just to be boring:

In [451]:
acs['io_ind'] = acs['io_ind'].apply(lambda d: [str(di) for di in d])

# Conclusion

I will save this frame. Once we have some results from the IO tables we can map the detailed industries to an output-weighted average of the industries in `io_ind` for each person in the ACS.

In [452]:
acs.to_pickle('../data/int/acs_to_io_crosswalk.pkl')